# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/02_metriques_evaluation/02_exercices_solutions.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('📦 Installation des packages...')
    !pip install -q numpy pandas matplotlib seaborn scikit-learn
    print('✅ Installation terminée !')
else:
    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 02 - Solutions : Métriques d'Évaluation

Ce notebook contient les solutions des exercices sur les métriques d'évaluation en Machine Learning.

**Objectifs** :
- Calculer et interpréter les métriques de classification
- Utiliser les métriques de régression
- Appliquer la validation croisée
- Choisir les bonnes métriques selon le contexte

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, precision_recall_curve,
    mean_squared_error, mean_absolute_error, r2_score,
    ConfusionMatrixDisplay
)

np.random.seed(42)
sns.set_style('whitegrid')

print("✅ Bibliothèques importées")

---

## Exercice 1 : Matrice de Confusion et Métriques de Base

**Objectif** : Calculer manuellement les métriques de classification à partir d'une matrice de confusion.

**Contexte** : Un système de détection de fraude bancaire a les résultats suivants :

```
Matrice de Confusion :
                Prédit Non-Fraude  Prédit Fraude
Réel Non-Fraude       950              50
Réel Fraude            20              80
```

**Questions** :

1. Identifiez les valeurs de TP, TN, FP, FN
2. Calculez manuellement :
   - Accuracy
   - Precision
   - Recall (Sensibilité)
   - Spécificité
   - F1-Score
3. Dans ce contexte de détection de fraude, quelle métrique est la plus importante ? Pourquoi ?
4. Le système est-il performant ? Justifiez votre réponse.

In [ ]:
# Solution

# Définissez la matrice de confusion
# Format scikit-learn : cm[i, j] où i = classe réelle, j = classe prédite
cm = np.array([
    [950, 50],   # Réel Non-Fraude (classe 0) : 950 correctement classés, 50 mal classés
    [20, 80]     # Réel Fraude (classe 1) : 20 mal classés, 80 correctement classés
])

print("Matrice de Confusion :")
print(cm)
print("")

# Question 1 : Identifiez TP, TN, FP, FN
# Pour la classe "Fraude" (classe positive) :
TN = cm[0, 0]  # Vrai Négatif : Réel Non-Fraude, Prédit Non-Fraude
FP = cm[0, 1]  # Faux Positif : Réel Non-Fraude, Prédit Fraude
FN = cm[1, 0]  # Faux Négatif : Réel Fraude, Prédit Non-Fraude
TP = cm[1, 1]  # Vrai Positif : Réel Fraude, Prédit Fraude

print("1. Identification des valeurs :")
print(f"   TP (Vrai Positif - fraude détectée) : {TP}")
print(f"   TN (Vrai Négatif - non-fraude détectée) : {TN}")
print(f"   FP (Faux Positif - fausse alerte) : {FP}")
print(f"   FN (Faux Négatif - fraude manquée) : {FN}")
print("")

# Question 2 : Calculez les métriques
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)  # Aussi appelée Sensibilité ou TPR (True Positive Rate)
specificity = TN / (TN + FP)  # TNR (True Negative Rate)
f1_score_manual = 2 * (precision * recall) / (precision + recall)

print("2. Métriques calculées manuellement :")
print(f"   Accuracy    : {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision   : {precision:.4f} ({precision*100:.2f}%)")
print(f"   Recall      : {recall:.4f} ({recall*100:.2f}%)")
print(f"   Specificity : {specificity:.4f} ({specificity*100:.2f}%)")
print(f"   F1-Score    : {f1_score_manual:.4f}")
print("")

# Interprétation de chaque métrique
print("   Interprétations :")
print(f"   - Accuracy : {accuracy*100:.1f}% des transactions sont correctement classifiées")
print(f"   - Precision : {precision*100:.1f}% des alertes de fraude sont réelles")
print(f"   - Recall : {recall*100:.1f}% des fraudes réelles sont détectées")
print(f"   - Specificity : {specificity*100:.1f}% des transactions légitimes sont correctement identifiées")
print("")

# Question 3 : Métrique la plus importante pour la détection de fraude
print("3. Métrique la plus importante pour la détection de fraude :")
print("")
print("   ➤ RECALL (Sensibilité) est la métrique la plus critique !")
print("")
print("   Justification :")
print("   - Il est CRUCIAL de détecter le maximum de fraudes (minimiser FN)")
print("   - Manquer une fraude (FN) peut coûter très cher à la banque")
print("   - Un faux positif (FP) est moins grave : on vérifie manuellement")
print("   - Mieux vaut trop d'alertes que pas assez en contexte de fraude")
print("")
print("   Dans ce cas : Recall = 80% signifie que 20% des fraudes ne sont PAS détectées")
print("   → C'est un problème majeur pour un système de détection de fraude !")
print("")

# Question 4 : Le système est-il performant ?
print("4. Évaluation globale du système :")
print("")
print("   ❌ Le système n'est PAS suffisamment performant")
print("")
print("   Problèmes identifiés :")
print(f"   - Recall de {recall*100:.1f}% est INSUFFISANT : 20 fraudes sur 100 passent inaperçues")
print(f"   - 20 fraudes manquées (FN) représentent un risque financier important")
print("")
print("   Points positifs :")
print(f"   - Precision de {precision*100:.1f}% est correcte : peu de fausses alertes")
print(f"   - Specificity de {specificity*100:.1f}% est excellente : peu de clients légitimes bloqués")
print("")
print("   Recommandations :")
print("   1. Ajuster le seuil de décision pour augmenter le Recall (accepter plus de FP)")
print("   2. Enrichir les features du modèle")
print("   3. Utiliser des techniques de rééquilibrage (SMOTE, class_weight)")
print("   4. Tester des modèles plus complexes (Random Forest, XGBoost)")
print("   5. Objectif : Atteindre au minimum 95% de Recall")

# Visualisation de la matrice de confusion
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Fraude', 'Fraude'],
            yticklabels=['Non-Fraude', 'Fraude'],
            cbar_kws={'label': 'Nombre de transactions'})
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Réelle')
plt.title('Matrice de Confusion - Détection de Fraude Bancaire')
plt.tight_layout()
plt.show()

# Visualisation comparative des métriques
metrics_names = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-Score']
metrics_values = [accuracy, precision, recall, specificity, f1_score_manual]

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if m >= 0.9 else 'orange' if m >= 0.8 else 'red' for m in metrics_values]
bars = ax.bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black')

# Ajouter les valeurs sur les barres
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.3f}',
            ha='center', va='bottom', fontweight='bold')

ax.axhline(y=0.95, color='green', linestyle='--', linewidth=2, label='Seuil excellent (95%)', alpha=0.5)
ax.axhline(y=0.80, color='orange', linestyle='--', linewidth=2, label='Seuil acceptable (80%)', alpha=0.5)
ax.set_ylim(0, 1.1)
ax.set_ylabel('Score')
ax.set_title('Comparaison des Métriques de Performance')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

## Exercice 2 : Courbe ROC et Choix du Seuil

**Objectif** : Comprendre l'impact du seuil de classification sur les performances.

**Consignes** :

1. Chargez le dataset Breast Cancer
2. Entraînez un modèle Logistic Regression
3. Tracez la courbe ROC et calculez l'AUC
4. Testez 3 seuils différents (0.3, 0.5, 0.7) et pour chacun :
   - Calculez Precision, Recall, F1-Score
   - Affichez la matrice de confusion
5. Quel seuil choisiriez-vous pour :
   - Minimiser les faux négatifs (ne pas manquer de malades) ?
   - Maximiser la précision globale ?
   - Obtenir un bon compromis ?

In [ ]:
# Solution

# 1. Chargez les données
data = load_breast_cancer()
X, y = data.data, data.target

print(f"Dataset Breast Cancer :")
print(f"  - Nombre d'échantillons : {X.shape[0]}")
print(f"  - Nombre de features : {X.shape[1]}")
print(f"  - Classes : {data.target_names}")
print(f"  - Distribution : {np.bincount(y)} (0=malin, 1=bénin)")
print("")

# 2. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Split train/test :")
print(f"  - Train : {X_train.shape[0]} échantillons")
print(f"  - Test : {X_test.shape[0]} échantillons")
print("")

# 3. Entraînez le modèle
model = LogisticRegression(max_iter=10000, random_state=42)
model.fit(X_train, y_train)

# Obtenir les probabilités prédites
y_proba = model.predict_proba(X_test)[:, 1]  # Probabilité de la classe positive (bénin)

print("✅ Modèle entraîné")
print("")

# 4. Tracez la courbe ROC
fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba)
auc = roc_auc_score(y_test, y_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Chance (AUC = 0.5)')
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Recall)', fontsize=12)
plt.title('Courbe ROC - Breast Cancer Classification', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"AUC Score : {auc:.4f}")
print(f"Interprétation : AUC proche de 1 indique un excellent modèle discriminant")
print("")

# 5. Testez différents seuils
thresholds_to_test = [0.3, 0.5, 0.7]

results = []
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, threshold in enumerate(thresholds_to_test):
    # Prédictions avec le seuil personnalisé
    y_pred = (y_proba >= threshold).astype(int)
    
    # Calculer les métriques
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    
    # Stocker les résultats
    results.append({
        'Threshold': threshold,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Accuracy': accuracy,
        'FN': cm[1, 0] if cm.shape[0] > 1 else 0  # Faux négatifs
    })
    
    # Visualiser la matrice de confusion
    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Malin', 'Bénin'],
                yticklabels=['Malin', 'Bénin'])
    ax.set_xlabel('Prédiction')
    ax.set_ylabel('Réalité')
    ax.set_title(f'Seuil = {threshold}\nF1={f1:.3f}, Recall={recall:.3f}')

plt.tight_layout()
plt.show()

# Afficher le tableau de comparaison
results_df = pd.DataFrame(results)
print("\nComparaison des seuils :")
print(results_df.to_string(index=False))
print("")

# Visualisation comparative
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(thresholds_to_test))
width = 0.2

ax.bar(x - width*1.5, results_df['Precision'], width, label='Precision', alpha=0.8)
ax.bar(x - width*0.5, results_df['Recall'], width, label='Recall', alpha=0.8)
ax.bar(x + width*0.5, results_df['F1-Score'], width, label='F1-Score', alpha=0.8)
ax.bar(x + width*1.5, results_df['Accuracy'], width, label='Accuracy', alpha=0.8)

ax.set_xlabel('Seuil')
ax.set_ylabel('Score')
ax.set_title('Impact du Seuil sur les Métriques de Performance')
ax.set_xticks(x)
ax.set_xticklabels([f'{t:.1f}' for t in thresholds_to_test])
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1.1)
plt.tight_layout()
plt.show()

# Répondre aux questions
print("\n" + "="*70)
print("RECOMMANDATIONS SELON L'OBJECTIF")
print("="*70)
print("")

print("1️⃣  Pour MINIMISER les faux négatifs (ne pas manquer de malades) :")
print(f"   → Choisir seuil = 0.3")
print(f"   → Recall maximal = {results_df.loc[0, 'Recall']:.3f}")
print(f"   → Seulement {results_df.loc[0, 'FN']:.0f} faux négatifs")
print(f"   → Justification : En médical, il vaut mieux sur-diagnostiquer que manquer un cancer")
print("")

print("2️⃣  Pour MAXIMISER la précision globale :")
print(f"   → Choisir seuil = 0.5 (défaut)")
print(f"   → Accuracy = {results_df.loc[1, 'Accuracy']:.3f}")
print(f"   → Bon équilibre entre toutes les métriques")
print("")

print("3️⃣  Pour un BON COMPROMIS (Precision-Recall) :")
print(f"   → Choisir seuil = 0.5")
print(f"   → F1-Score maximal = {results_df.loc[1, 'F1-Score']:.3f}")
print(f"   → Meilleur équilibre entre Precision ({results_df.loc[1, 'Precision']:.3f}) et Recall ({results_df.loc[1, 'Recall']:.3f})")
print("")

print("💡 CONSEIL GÉNÉRAL :")
print("   En contexte médical, PRIVILÉGIER LE RECALL pour ne manquer aucun cas grave.")
print("   → Recommandation finale : Seuil = 0.3 ou 0.4")

---

## Exercice 3 : Métriques de Régression

**Objectif** : Évaluer un modèle de régression avec différentes métriques.

**Consignes** :

1. Chargez le dataset Diabetes
2. Entraînez deux modèles :
   - LinearRegression
   - RandomForestRegressor
3. Pour chaque modèle, calculez :
   - MSE (Mean Squared Error)
   - RMSE (Root Mean Squared Error)
   - MAE (Mean Absolute Error)
   - R² (Coefficient de détermination)
4. Comparez les deux modèles
5. Visualisez les prédictions vs valeurs réelles pour le meilleur modèle

In [ ]:
# Solution

# 1. Chargez le dataset Diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

print("Dataset Diabetes :")
print(f"  - Échantillons : {X.shape[0]}")
print(f"  - Features : {X.shape[1]}")
print(f"  - Target : Progression du diabète (valeur continue)")
print(f"  - Range target : [{y.min():.1f}, {y.max():.1f}]")
print("")

# 2. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 3. Entraînez les deux modèles
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
}

results = []

for name, model in models.items():
    print(f"Entraînement de {name}...")
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calcul des métriques sur le test set
    mse = mean_squared_error(y_test, y_pred_test)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    
    # Métriques sur train (pour détecter l'overfitting)
    r2_train = r2_score(y_train, y_pred_train)
    
    results.append({
        'Model': name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R² (test)': r2,
        'R² (train)': r2_train,
        'Predictions': y_pred_test
    })
    
    print(f"  ✓ {name} entraîné")
    print("")

# 4. Comparer les modèles
print("="*70)
print("COMPARAISON DES MODÈLES")
print("="*70)
print("")

# Créer un DataFrame pour la comparaison
comparison_df = pd.DataFrame(results)[['Model', 'MSE', 'RMSE', 'MAE', 'R² (test)', 'R² (train)']]
print(comparison_df.to_string(index=False))
print("")

# Interprétation des métriques
print("INTERPRÉTATION DES MÉTRIQUES :")
print("")
print("1. MSE (Mean Squared Error) :")
print("   - Pénalise fortement les grandes erreurs (au carré)")
print("   - Plus le MSE est bas, meilleur est le modèle")
print("   - Unité : carré de l'unité de y (difficile à interpréter directement)")
print("")
print("2. RMSE (Root Mean Squared Error) :")
print("   - Racine carrée du MSE")
print("   - Même unité que y (plus interprétable)")
print("   - Représente l'erreur moyenne en unités originales")
print("")
print("3. MAE (Mean Absolute Error) :")
print("   - Moyenne des valeurs absolues des erreurs")
print("   - Moins sensible aux outliers que MSE/RMSE")
print("   - Même unité que y")
print("")
print("4. R² (Coefficient de détermination) :")
print("   - Mesure la proportion de variance expliquée par le modèle")
print("   - Varie entre -∞ et 1 (1 = parfait, 0 = modèle constant)")
print("   - R² train vs test permet de détecter l'overfitting")
print("")

# Déterminer le meilleur modèle
best_idx = comparison_df['R² (test)'].idxmax()
best_model_name = comparison_df.loc[best_idx, 'Model']
best_r2 = comparison_df.loc[best_idx, 'R² (test)']

print(f"🏆 MEILLEUR MODÈLE : {best_model_name}")
print(f"   R² = {best_r2:.4f}")
print("")

# Analyse de l'overfitting
for idx, row in comparison_df.iterrows():
    diff = row['R² (train)'] - row['R² (test)']
    print(f"{row['Model']} :")
    print(f"  R² train = {row['R² (train)']:.4f}")
    print(f"  R² test  = {row['R² (test)']:.4f}")
    print(f"  Écart    = {diff:.4f}")
    if diff > 0.1:
        print(f"  ⚠️  Overfitting détecté !")
    else:
        print(f"  ✓ Pas d'overfitting majeur")
    print("")

# 5. Visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# a) Comparaison visuelle des métriques
ax1 = axes[0, 0]
metrics = ['MSE', 'RMSE', 'MAE']
x = np.arange(len(metrics))
width = 0.35

for i, result in enumerate(results):
    values = [result[m] for m in metrics]
    ax1.bar(x + i*width, values, width, label=result['Model'], alpha=0.8)

ax1.set_xlabel('Métriques')
ax1.set_ylabel('Erreur')
ax1.set_title('Comparaison des Erreurs')
ax1.set_xticks(x + width / 2)
ax1.set_xticklabels(metrics)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# b) Comparaison R²
ax2 = axes[0, 1]
model_names = [r['Model'] for r in results]
r2_train_values = [r['R² (train)'] for r in results]
r2_test_values = [r['R² (test)'] for r in results]

x = np.arange(len(model_names))
ax2.bar(x - width/2, r2_train_values, width, label='R² Train', alpha=0.8)
ax2.bar(x + width/2, r2_test_values, width, label='R² Test', alpha=0.8)
ax2.set_xlabel('Modèles')
ax2.set_ylabel('R² Score')
ax2.set_title('Comparaison R² (Train vs Test)')
ax2.set_xticks(x)
ax2.set_xticklabels(model_names, rotation=15, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)
ax2.set_ylim(0, 1)

# c) Prédictions vs Réalité pour Linear Regression
ax3 = axes[1, 0]
y_pred_lr = results[0]['Predictions']
ax3.scatter(y_test, y_pred_lr, alpha=0.6, edgecolors='k', linewidth=0.5)
ax3.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Prédiction parfaite')
ax3.set_xlabel('Valeurs Réelles')
ax3.set_ylabel('Prédictions')
ax3.set_title(f'Linear Regression (R²={results[0]["R² (test)"]:.3f})')
ax3.legend()
ax3.grid(alpha=0.3)

# d) Prédictions vs Réalité pour Random Forest
ax4 = axes[1, 1]
y_pred_rf = results[1]['Predictions']
ax4.scatter(y_test, y_pred_rf, alpha=0.6, edgecolors='k', linewidth=0.5, color='green')
ax4.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Prédiction parfaite')
ax4.set_xlabel('Valeurs Réelles')
ax4.set_ylabel('Prédictions')
ax4.set_title(f'Random Forest (R²={results[1]["R² (test)"]:.3f})')
ax4.legend()
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Distribution des résidus pour le meilleur modèle
best_predictions = results[best_idx]['Predictions']
residuals = y_test - best_predictions

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme des résidus
axes[0].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Résidu nul')
axes[0].set_xlabel('Résidus (Réel - Prédit)')
axes[0].set_ylabel('Fréquence')
axes[0].set_title(f'Distribution des Résidus - {best_model_name}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Résidus vs Prédictions
axes[1].scatter(best_predictions, residuals, alpha=0.6, edgecolors='k', linewidth=0.5)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2, label='Résidu nul')
axes[1].set_xlabel('Prédictions')
axes[1].set_ylabel('Résidus')
axes[1].set_title(f'Résidus vs Prédictions - {best_model_name}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("CONCLUSION")
print("="*70)
print("")
print(f"Le modèle {best_model_name} performe mieux avec un R² de {best_r2:.4f}.")
print("")
print("Observations :")
print("- Random Forest capture mieux les relations non-linéaires")
print("- Les résidus devraient idéalement être centrés sur 0 et distribués normalement")
print("- Un R² autour de 0.4-0.5 indique que le modèle explique environ 40-50% de la variance")
print("- Il reste de la marge d'amélioration (feature engineering, hyperparameter tuning)")

---

## Exercice 4 : Validation Croisée

**Objectif** : Comparer la validation simple (train/test split) avec la validation croisée.

**Consignes** :

1. Utilisez le dataset Breast Cancer
2. **Méthode 1** : Entraînez un RandomForestClassifier avec un simple split 70/30
   - Calculez l'accuracy sur le test set
3. **Méthode 2** : Utilisez la validation croisée 5-fold (`cross_val_score`)
   - Calculez la moyenne et l'écart-type des scores
4. **Méthode 3** : Testez différents nombres de folds (3, 5, 10, 20)
   - Visualisez l'impact sur la variance des résultats
5. Quelle méthode vous semble la plus fiable ? Pourquoi ?
6. Quel nombre de folds recommanderiez-vous ?

In [ ]:
# Solution

# 1. Chargez les données
data = load_breast_cancer()
X, y = data.data, data.target

print("Dataset Breast Cancer pour validation croisée")
print(f"  - Total échantillons : {len(X)}")
print("")

# 2. Méthode 1 : Simple train/test split
print("="*70)
print("MÉTHODE 1 : SIMPLE TRAIN/TEST SPLIT (70/30)")
print("="*70)
print("")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model_simple = RandomForestClassifier(n_estimators=100, random_state=42)
model_simple.fit(X_train, y_train)
accuracy_simple = model_simple.score(X_test, y_test)

print(f"Accuracy sur le test set : {accuracy_simple:.4f}")
print("")
print("Limites de cette méthode :")
print("  ❌ Dépend fortement du split aléatoire")
print("  ❌ Pas d'estimation de la variance")
print("  ❌ Une partie des données (30%) n'est jamais utilisée pour l'entraînement")
print("")

# 3. Méthode 2 : Validation croisée 5-fold
print("="*70)
print("MÉTHODE 2 : VALIDATION CROISÉE 5-FOLD")
print("="*70)
print("")

model_cv = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores_5 = cross_val_score(model_cv, X, y, cv=5, scoring='accuracy')

print(f"Scores des 5 folds : {cv_scores_5}")
print(f"Moyenne : {cv_scores_5.mean():.4f}")
print(f"Écart-type : {cv_scores_5.std():.4f}")
print(f"Intervalle de confiance (±2σ) : [{cv_scores_5.mean() - 2*cv_scores_5.std():.4f}, {cv_scores_5.mean() + 2*cv_scores_5.std():.4f}]")
print("")
print("Avantages :")
print("  ✅ Tous les exemples sont utilisés pour train ET test")
print("  ✅ Estimation de la variance du modèle")
print("  ✅ Résultat plus fiable et robuste")
print("")

# 4. Méthode 3 : Tester différents nombres de folds
print("="*70)
print("MÉTHODE 3 : IMPACT DU NOMBRE DE FOLDS")
print("="*70)
print("")

n_folds_list = [3, 5, 10, 20]
cv_results = []

for n_folds in n_folds_list:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(model, X, y, cv=n_folds, scoring='accuracy')
    
    cv_results.append({
        'n_folds': n_folds,
        'mean': scores.mean(),
        'std': scores.std(),
        'min': scores.min(),
        'max': scores.max(),
        'scores': scores
    })
    
    print(f"{n_folds}-fold CV :")
    print(f"  Moyenne : {scores.mean():.4f}")
    print(f"  Écart-type : {scores.std():.4f}")
    print(f"  Range : [{scores.min():.4f}, {scores.max():.4f}]")
    print("")

# Visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# a) Boxplot des scores selon le nombre de folds
ax1 = axes[0, 0]
data_for_boxplot = [result['scores'] for result in cv_results]
bp = ax1.boxplot(data_for_boxplot, labels=[f"{nf}-fold" for nf in n_folds_list],
                  patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
ax1.set_xlabel('Nombre de Folds')
ax1.set_ylabel('Accuracy')
ax1.set_title('Distribution des Scores selon le Nombre de Folds')
ax1.grid(axis='y', alpha=0.3)

# b) Moyenne et écart-type
ax2 = axes[0, 1]
means = [r['mean'] for r in cv_results]
stds = [r['std'] for r in cv_results]
ax2.errorbar(n_folds_list, means, yerr=stds, marker='o', markersize=8,
             capsize=5, capthick=2, linewidth=2, label='Moyenne ± Écart-type')
ax2.axhline(accuracy_simple, color='red', linestyle='--', linewidth=2,
            label=f'Simple split ({accuracy_simple:.4f})')
ax2.set_xlabel('Nombre de Folds')
ax2.set_ylabel('Accuracy')
ax2.set_title('Moyenne et Variance selon le Nombre de Folds')
ax2.legend()
ax2.grid(alpha=0.3)

# c) Écart-type uniquement
ax3 = axes[1, 0]
ax3.plot(n_folds_list, stds, marker='o', markersize=8, linewidth=2, color='orange')
ax3.set_xlabel('Nombre de Folds')
ax3.set_ylabel('Écart-type')
ax3.set_title('Stabilité du Modèle (Écart-type) vs Nombre de Folds')
ax3.grid(alpha=0.3)

# d) Scores individuels pour 5-fold CV
ax4 = axes[1, 1]
for i, result in enumerate(cv_results):
    n_folds = result['n_folds']
    scores = result['scores']
    ax4.scatter([n_folds] * len(scores), scores, alpha=0.6, s=50, label=f"{n_folds}-fold")
ax4.set_xlabel('Nombre de Folds')
ax4.set_ylabel('Accuracy')
ax4.set_title('Scores Individuels de Chaque Fold')
ax4.legend()
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 5 & 6. Recommandations
print("\n" + "="*70)
print("RECOMMANDATIONS")
print("="*70)
print("")

print("5️⃣  Quelle méthode est la plus fiable ?")
print("")
print("   → LA VALIDATION CROISÉE est NETTEMENT plus fiable !")
print("")
print("   Raisons :")
print("   ✅ Utilise toutes les données (train + test)")
print("   ✅ Fournit une estimation de la variance")
print("   ✅ Réduit le biais lié à un split particulier")
print("   ✅ Plus robuste pour évaluer la généralisation")
print("")

print("6️⃣  Nombre de folds recommandé ?")
print("")
print("   → RECOMMANDATION : 5 ou 10 folds")
print("")
print("   Compromis :")
print("   • 3 folds : Rapide, mais variance élevée")
print("   • 5 folds : BON COMPROMIS - Standard en pratique")
print("   • 10 folds : Estimation plus précise, temps de calcul acceptable")
print("   • 20 folds : Très précis, mais coûteux en temps (et variance peut augmenter)")
print("")
print("   Règle générale :")
print("   - Dataset petit (<1000 exemples) : 10-fold ou LOOCV")
print("   - Dataset moyen : 5-fold (standard)")
print("   - Dataset large (>100k exemples) : 3-fold ou simple split suffit")
print("")

print("💡 CONSEIL FINAL :")
print("   Utilisez TOUJOURS la validation croisée pour évaluer vos modèles.")
print("   Le simple train/test split ne devrait être utilisé que pour :")
print("   - Des datasets très larges (>1M d'exemples)")
print("   - Des prototypes rapides")
print("   - Le set de test FINAL (après sélection du modèle)")

---

## Exercice 5 : Cas Pratique - Système de Recommandation de Films

**Objectif** : Choisir les bonnes métriques selon le contexte métier.

**Contexte** : Vous développez un système de recommandation de films. Vous avez deux versions du modèle avec les résultats suivants sur 1000 utilisateurs :

**Modèle A** :
- Films recommandés et regardés (TP) : 400
- Films recommandés mais non regardés (FP) : 100
- Films non recommandés mais regardés (FN) : 200
- Films non recommandés et non regardés (TN) : 300

**Modèle B** :
- Films recommandés et regardés (TP) : 500
- Films recommandés mais non regardés (FP) : 300
- Films non recommandés mais regardés (FN) : 100
- Films non recommandés et non regardés (TN) : 100

**Questions** :

1. Calculez pour chaque modèle : Accuracy, Precision, Recall, F1-Score
2. Quel modèle choisiriez-vous si :
   - Vous voulez éviter de recommander de mauvais films (utilisateur satisfait) ?
   - Vous voulez maximiser la découverte de films (ne pas manquer de bons films) ?
   - Vous voulez un compromis équilibré ?
3. Quelle métrique est la plus pertinente pour un système de recommandation ? Justifiez.
4. Créez une visualisation comparative des deux modèles

In [ ]:
# Solution

# 1. Définissez les matrices de confusion
models_data = {
    'Modèle A': {'TP': 400, 'FP': 100, 'FN': 200, 'TN': 300},
    'Modèle B': {'TP': 500, 'FP': 300, 'FN': 100, 'TN': 100}
}

# Calculer les métriques pour chaque modèle
results = []

for model_name, data in models_data.items():
    TP, FP, FN, TN = data['TP'], data['FP'], data['FN'], data['TN']
    
    # Calcul des métriques
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Matrice de confusion
    cm = np.array([[TN, FP], [FN, TP]])
    
    results.append({
        'Model': model_name,
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'CM': cm
    })

# Afficher les résultats
print("="*70)
print("1. MÉTRIQUES CALCULÉES")
print("="*70)
print("")

comparison_df = pd.DataFrame(results)[['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'TP', 'FP', 'FN', 'TN']]
print(comparison_df.to_string(index=False))
print("")

# 2. Analyse selon les objectifs métier
print("="*70)
print("2. CHOIX DU MODÈLE SELON L'OBJECTIF MÉTIER")
print("="*70)
print("")

print("❶ Éviter de recommander de MAUVAIS films (utilisateur satisfait) :")
print("")
print("   → Métrique clé : PRECISION")
print("   → Objectif : Minimiser les FP (recommandations non pertinentes)")
print("")
best_precision_idx = comparison_df['Precision'].idxmax()
best_precision_model = comparison_df.loc[best_precision_idx, 'Model']
best_precision_value = comparison_df.loc[best_precision_idx, 'Precision']
print(f"   ✅ CHOIX : {best_precision_model}")
print(f"   Precision = {best_precision_value:.1%}")
print(f"   → {best_precision_value*100:.0f}% des films recommandés sont effectivement regardés")
print("   → Moins de frustration utilisateur")
print("")

print("❷ Maximiser la DÉCOUVERTE de films (ne pas manquer de bons films) :")
print("")
print("   → Métrique clé : RECALL")
print("   → Objectif : Minimiser les FN (opportunités manquées)")
print("")
best_recall_idx = comparison_df['Recall'].idxmax()
best_recall_model = comparison_df.loc[best_recall_idx, 'Model']
best_recall_value = comparison_df.loc[best_recall_idx, 'Recall']
print(f"   ✅ CHOIX : {best_recall_model}")
print(f"   Recall = {best_recall_value:.1%}")
print(f"   → {best_recall_value*100:.0f}% des films potentiellement intéressants sont recommandés")
print("   → Maximise l'engagement utilisateur")
print("")

print("❸ Compromis ÉQUILIBRÉ :")
print("")
print("   → Métrique clé : F1-SCORE")
print("   → Objectif : Balance entre Precision et Recall")
print("")
best_f1_idx = comparison_df['F1-Score'].idxmax()
best_f1_model = comparison_df.loc[best_f1_idx, 'Model']
best_f1_value = comparison_df.loc[best_f1_idx, 'F1-Score']
print(f"   ✅ CHOIX : {best_f1_model}")
print(f"   F1-Score = {best_f1_value:.3f}")
print("   → Meilleur équilibre qualité/quantité des recommandations")
print("")

# 3. Métrique la plus pertinente
print("="*70)
print("3. MÉTRIQUE LA PLUS PERTINENTE POUR LA RECOMMANDATION")
print("="*70)
print("")
print("   → RECALL est généralement PLUS IMPORTANT en recommandation !")
print("")
print("   Justification :")
print("   📌 Objectif principal : Maximiser l'engagement et la satisfaction")
print("   📌 Il vaut mieux recommander plus (quitte à avoir quelques ratés)")
print("      que de manquer des films que l'utilisateur aurait adoré")
print("   📌 Un FP (recommandation non regardée) est moins grave qu'un FN (opportunité manquée)")
print("   📌 L'utilisateur peut facilement ignorer une recommandation")
print("   📌 Mais il ne saura jamais ce qu'il a manqué (FN)")
print("")
print("   ⚠️  MAIS : Trop de FP peut nuire à la confiance dans le système")
print("   → Solution : Trouver le bon compromis via F1-Score ou F2-Score")
print("   → F2-Score = met plus de poids sur le Recall que la Precision")
print("")

# Calculer F2-Score
for result in results:
    precision = result['Precision']
    recall = result['Recall']
    beta = 2
    f2 = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall) if (precision + recall) > 0 else 0
    result['F2-Score'] = f2

print("   Comparaison F1 vs F2 (F2 favorise le Recall) :")
for result in results:
    print(f"   {result['Model']} : F1={result['F1-Score']:.3f}, F2={result['F2-Score']:.3f}")
print("")

# 4. Visualisations
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# a) Matrices de confusion côte à côte
for idx, result in enumerate(results):
    ax = fig.add_subplot(gs[0, idx])
    cm = result['CM']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Non Regardé', 'Regardé'],
                yticklabels=['Non Recommandé', 'Recommandé'])
    ax.set_xlabel('Réalité')
    ax.set_ylabel('Recommandation')
    ax.set_title(f'{result["Model"]} - Matrice de Confusion')

# b) Comparaison des métriques principales
ax3 = fig.add_subplot(gs[1, :])
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics_to_plot))
width = 0.35

for i, result in enumerate(results):
    values = [result[m] for m in metrics_to_plot]
    ax3.bar(x + i*width, values, width, label=result['Model'], alpha=0.8)
    # Ajouter les valeurs sur les barres
    for j, v in enumerate(values):
        ax3.text(x[j] + i*width, v + 0.02, f'{v:.2f}', ha='center', fontsize=9, fontweight='bold')

ax3.set_xlabel('Métriques', fontsize=12)
ax3.set_ylabel('Score', fontsize=12)
ax3.set_title('Comparaison des Métriques de Performance', fontsize=14)
ax3.set_xticks(x + width / 2)
ax3.set_xticklabels(metrics_to_plot)
ax3.legend(fontsize=11)
ax3.grid(axis='y', alpha=0.3)
ax3.set_ylim(0, 1.1)

# c) Radar chart pour visualisation multidimensionnelle
ax4 = fig.add_subplot(gs[2, :], projection='polar')

categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
N = len(categories)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

for result in results:
    values = [result[m] for m in categories]
    values += values[:1]
    ax4.plot(angles, values, 'o-', linewidth=2, label=result['Model'])
    ax4.fill(angles, values, alpha=0.15)

ax4.set_xticks(angles[:-1])
ax4.set_xticklabels(categories)
ax4.set_ylim(0, 1)
ax4.set_title('Comparaison Multi-Métriques (Radar Chart)', fontsize=14, pad=20)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax4.grid(True)

plt.show()

# Tableau récapitulatif des recommandations
print("="*70)
print("RECOMMANDATION FINALE")
print("="*70)
print("")
print(f"Pour un système de recommandation de films, nous recommandons : {best_recall_model}")
print("")
print("Raisons :")
print(f"  ✓ Recall élevé ({best_recall_value:.1%}) : Capture la majorité des films pertinents")
print("  ✓ Maximise l'engagement utilisateur")
print("  ✓ Réduit les opportunités manquées")
print("")
print("Trade-offs acceptés :")
modelB_precision = comparison_df.loc[comparison_df['Model'] == best_recall_model, 'Precision'].values[0]
print(f"  • Precision légèrement plus basse ({modelB_precision:.1%})")
print("  • Mais acceptable car les utilisateurs peuvent ignorer les recommandations non pertinentes")
print("")
print("💡 Pour améliorer encore :")
print("  1. Implémenter un système de ranking (ne pas montrer toutes les recommandations)")
print("  2. Utiliser un feedback utilisateur pour ajuster le seuil")
print("  3. Segmenter les utilisateurs (certains préfèrent qualité, d'autres quantité)")

---

## Exercice 6 : Dataset Déséquilibré

**Objectif** : Comprendre l'impact du déséquilibre des classes sur les métriques.

**Consignes** :

1. Créez un dataset déséquilibré synthétique :
   - Classe 0 (majoritaire) : 950 exemples
   - Classe 1 (minoritaire) : 50 exemples
2. Entraînez un modèle simple (LogisticRegression)
3. Calculez et comparez :
   - Accuracy
   - Precision, Recall, F1 pour chaque classe
   - Courbe ROC et AUC
   - Courbe Precision-Recall
4. Que se passe-t-il si le modèle prédit toujours la classe majoritaire ?
5. Quelle métrique est la plus révélatrice du problème ?
6. **Bonus** : Testez une technique de rééquilibrage (SMOTE ou class_weight)

In [ ]:
# Solution
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

# 1. Créez un dataset déséquilibré
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    weights=[0.95, 0.05],  # 95% classe 0, 5% classe 1
    random_state=42,
    flip_y=0.01  # Légère noise
)

print("Dataset déséquilibré créé :")
print(f"  Total échantillons : {len(y)}")
print(f"  Distribution : {np.bincount(y)}")
print(f"  Ratio : {np.bincount(y)[0]/len(y):.1%} classe 0, {np.bincount(y)[1]/len(y):.1%} classe 1")
print("")

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Distribution train : {np.bincount(y_train)}")
print(f"Distribution test : {np.bincount(y_test)}")
print("")

# 2. Entraînez un modèle simple sans traitement du déséquilibre
print("="*70)
print("MODÈLE 1 : SANS TRAITEMENT DU DÉSÉQUILIBRE")
print("="*70)
print("")

model_imbalanced = LogisticRegression(max_iter=1000, random_state=42)
model_imbalanced.fit(X_train, y_train)
y_pred_imbalanced = model_imbalanced.predict(X_test)
y_proba_imbalanced = model_imbalanced.predict_proba(X_test)[:, 1]

# 3. Calculez les métriques
accuracy_imb = accuracy_score(y_test, y_pred_imbalanced)
cm_imb = confusion_matrix(y_test, y_pred_imbalanced)

print("Matrice de confusion :")
print(cm_imb)
print("")

print(f"Accuracy : {accuracy_imb:.4f}")
print("")

# Classification report détaillé
print("Classification Report :")
print(classification_report(y_test, y_pred_imbalanced, target_names=['Classe 0 (maj)', 'Classe 1 (min)']))

# 4. Modèle naïf : toujours prédire la classe majoritaire
print("="*70)
print("MODÈLE NAÏF : TOUJOURS PRÉDIRE CLASSE 0 (MAJORITAIRE)")
print("="*70)
print("")

y_pred_naive = np.zeros_like(y_test)  # Toujours prédire 0
accuracy_naive = accuracy_score(y_test, y_pred_naive)
cm_naive = confusion_matrix(y_test, y_pred_naive)

print("Matrice de confusion :")
print(cm_naive)
print("")

print(f"Accuracy : {accuracy_naive:.4f}")
print("")

print("⚠️  OBSERVATION CRITIQUE :")
print(f"   Un modèle qui prédit TOUJOURS la classe majoritaire obtient {accuracy_naive:.1%} d'accuracy !")
print("   → L'accuracy est une métrique TROMPEUSE pour les datasets déséquilibrés")
print("   → Le modèle n'a RIEN appris, mais semble performant")
print("")

# 5. Courbes ROC et Precision-Recall
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba_imbalanced)
roc_auc = roc_auc_score(y_test, y_proba_imbalanced)

ax1 = axes[0, 0]
ax1.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC={roc_auc:.3f})')
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Chance')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Courbe ROC')
ax1.legend()
ax1.grid(alpha=0.3)

# Precision-Recall Curve
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba_imbalanced)

ax2 = axes[0, 1]
ax2.plot(recall_vals, precision_vals, linewidth=2, color='orange')
baseline = np.sum(y_test) / len(y_test)  # Proportion de la classe minoritaire
ax2.axhline(baseline, color='k', linestyle='--', linewidth=1, label=f'Baseline ({baseline:.3f})')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Courbe Precision-Recall')
ax2.legend()
ax2.grid(alpha=0.3)

# Matrice de confusion
ax3 = axes[0, 2]
sns.heatmap(cm_imb, annot=True, fmt='d', cmap='Blues', ax=ax3,
            xticklabels=['Classe 0', 'Classe 1'],
            yticklabels=['Classe 0', 'Classe 1'])
ax3.set_xlabel('Prédiction')
ax3.set_ylabel('Réalité')
ax3.set_title('Matrice de Confusion - Modèle déséquilibré')

# 6. BONUS : Technique de rééquilibrage avec class_weight
print("="*70)
print("MODÈLE 2 : AVEC RÉÉQUILIBRAGE (class_weight='balanced')")
print("="*70)
print("")

model_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
model_balanced.fit(X_train, y_train)
y_pred_balanced = model_balanced.predict(X_test)
y_proba_balanced = model_balanced.predict_proba(X_test)[:, 1]

accuracy_bal = accuracy_score(y_test, y_pred_balanced)
cm_bal = confusion_matrix(y_test, y_pred_balanced)

print("Matrice de confusion :")
print(cm_bal)
print("")

print(f"Accuracy : {accuracy_bal:.4f}")
print("")

print("Classification Report :")
print(classification_report(y_test, y_pred_balanced, target_names=['Classe 0 (maj)', 'Classe 1 (min)']))

# Courbes pour le modèle balanced
fpr_bal, tpr_bal, _ = roc_curve(y_test, y_proba_balanced)
roc_auc_bal = roc_auc_score(y_test, y_proba_balanced)

ax4 = axes[1, 0]
ax4.plot(fpr, tpr, linewidth=2, label=f'Sans équilibrage (AUC={roc_auc:.3f})', alpha=0.7)
ax4.plot(fpr_bal, tpr_bal, linewidth=2, label=f'Avec équilibrage (AUC={roc_auc_bal:.3f})')
ax4.plot([0, 1], [0, 1], 'k--', linewidth=1)
ax4.set_xlabel('False Positive Rate')
ax4.set_ylabel('True Positive Rate')
ax4.set_title('Comparaison ROC')
ax4.legend()
ax4.grid(alpha=0.3)

precision_vals_bal, recall_vals_bal, _ = precision_recall_curve(y_test, y_proba_balanced)

ax5 = axes[1, 1]
ax5.plot(recall_vals, precision_vals, linewidth=2, label='Sans équilibrage', alpha=0.7, color='orange')
ax5.plot(recall_vals_bal, precision_vals_bal, linewidth=2, label='Avec équilibrage', color='green')
ax5.axhline(baseline, color='k', linestyle='--', linewidth=1, label=f'Baseline ({baseline:.3f})')
ax5.set_xlabel('Recall')
ax5.set_ylabel('Precision')
ax5.set_title('Comparaison Precision-Recall')
ax5.legend()
ax5.grid(alpha=0.3)

ax6 = axes[1, 2]
sns.heatmap(cm_bal, annot=True, fmt='d', cmap='Greens', ax=ax6,
            xticklabels=['Classe 0', 'Classe 1'],
            yticklabels=['Classe 0', 'Classe 1'])
ax6.set_xlabel('Prédiction')
ax6.set_ylabel('Réalité')
ax6.set_title('Matrice de Confusion - Modèle équilibré')

plt.tight_layout()
plt.show()

# Comparaison finale
print("\n" + "="*70)
print("ANALYSE COMPARATIVE")
print("="*70)
print("")

# Extraire les métriques pour la classe minoritaire
from sklearn.metrics import precision_recall_fscore_support

prec_imb, rec_imb, f1_imb, _ = precision_recall_fscore_support(y_test, y_pred_imbalanced, average=None)
prec_bal, rec_bal, f1_bal, _ = precision_recall_fscore_support(y_test, y_pred_balanced, average=None)

comparison = pd.DataFrame({
    'Métrique': ['Accuracy', 'Precision (classe 1)', 'Recall (classe 1)', 'F1-Score (classe 1)', 'ROC AUC'],
    'Sans équilibrage': [accuracy_imb, prec_imb[1], rec_imb[1], f1_imb[1], roc_auc],
    'Avec équilibrage': [accuracy_bal, prec_bal[1], rec_bal[1], f1_bal[1], roc_auc_bal],
    'Modèle naïf': [accuracy_naive, 0, 0, 0, 0.5]
})

print(comparison.to_string(index=False))
print("")

print("OBSERVATIONS CLÉS :")
print("")
print("1️⃣  L'Accuracy est TROMPEUSE :")
print(f"   - Modèle naïf : {accuracy_naive:.1%} (prédit toujours classe 0)")
print(f"   - Sans équilibrage : {accuracy_imb:.1%}")
print(f"   - Avec équilibrage : {accuracy_bal:.1%} (semble pire, mais plus informatif !)")
print("")
print("2️⃣  Le Recall de la classe minoritaire est CRITIQUE :")
print(f"   - Sans équilibrage : {rec_imb[1]:.1%} (manque beaucoup d'exemples)")
print(f"   - Avec équilibrage : {rec_bal[1]:.1%} (bien meilleur !)")
print("")
print("3️⃣  La courbe Precision-Recall est PLUS INFORMATIVE que ROC :")
print("   - ROC peut être optimiste sur datasets déséquilibrés")
print("   - Precision-Recall montre mieux les performances sur la classe minoritaire")
print("")
print("4️⃣  Le F1-Score de la classe minoritaire est le meilleur indicateur :")
print(f"   - Sans équilibrage : {f1_imb[1]:.3f}")
print(f"   - Avec équilibrage : {f1_bal[1]:.3f}")
print("")

print("💡 RECOMMANDATIONS pour datasets déséquilibrés :")
print("")
print("1. NE PAS utiliser l'Accuracy seule")
print("2. Privilégier : Precision, Recall, F1-Score de la classe minoritaire")
print("3. Utiliser la courbe Precision-Recall plutôt que ROC")
print("4. Techniques de rééquilibrage :")
print("   - class_weight='balanced' (simple et efficace)")
print("   - SMOTE (sur-échantillonnage synthétique)")
print("   - Sous-échantillonnage de la classe majoritaire")
print("   - Ensembles avec rééquilibrage (BalancedRandomForest)")
print("5. Ajuster le seuil de décision selon le coût métier")

---

## Exercice 7 : Bonus - Métriques Personnalisées

**Objectif** : Créer une métrique personnalisée adaptée à un contexte spécifique.

**Contexte** : Une entreprise de e-commerce veut minimiser les retours produits. Un retour produit coûte 20€ à l'entreprise, tandis qu'une vente réussie rapporte 10€.

**Consignes** :

1. Créez une fonction de coût personnalisée qui calcule le bénéfice/perte total
2. Utilisez cette fonction pour évaluer plusieurs modèles
3. Comparez avec les métriques classiques (accuracy, F1)
4. Quel modèle choisiriez-vous selon la métrique de coût ?

In [ ]:
# Solution

def custom_cost_metric(y_true, y_pred, profit_per_sale=10, cost_per_return=20):
    """
    Calcule le bénéfice total selon un modèle métier.
    
    Matrice de coût :
    - TP (vente réussie, bien prédite) : +10€
    - TN (pas de vente, bien prédite) : 0€
    - FP (vente avec retour non anticipé) : -20€ (le pire cas !)
    - FN (vente manquée, opportunité perdue) : 0€ (pas de perte directe)
    
    Args:
        y_true : Labels réels (1 = vente réussie, 0 = retour)
        y_pred : Prédictions (1 = recommander, 0 = ne pas recommander)
        profit_per_sale : Bénéfice par vente réussie
        cost_per_return : Coût d'un retour produit
    
    Returns:
        total_profit : Bénéfice total en €
    """
    cm = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cm.ravel()
    
    # Calcul du profit total
    profit_from_tp = TP * profit_per_sale  # Ventes réussies bien prédites
    cost_from_fp = FP * cost_per_return    # Retours non anticipés (coût élevé)
    
    total_profit = profit_from_tp - cost_from_fp
    
    return total_profit, {'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN}

# Créer un dataset synthétique pour le e-commerce
np.random.seed(42)
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=7,
    n_redundant=2,
    weights=[0.3, 0.7],  # 70% de ventes réussies, 30% de retours
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Dataset E-Commerce :")
print(f"  Total : {len(y)} transactions")
print(f"  Distribution : {np.bincount(y)[0]} retours, {np.bincount(y)[1]} ventes réussies")
print(f"  Taux de retour : {np.bincount(y)[0]/len(y):.1%}")
print("")

# Tester différents modèles avec différents hyperparamètres
models_configs = [
    ('Logistic (Conservative)', LogisticRegression(C=0.1, random_state=42)),
    ('Logistic (Standard)', LogisticRegression(C=1.0, random_state=42)),
    ('Logistic (Aggressive)', LogisticRegression(C=10.0, random_state=42)),
    ('RF (Conservative)', RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42)),
    ('RF (Standard)', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)),
]

results = []

print("="*70)
print("ÉVALUATION DES MODÈLES")
print("="*70)
print("")

for name, model in models_configs:
    # Entraîner
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Métriques classiques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Métrique personnalisée
    profit, components = custom_cost_metric(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Profit (€)': profit,
        'TP': components['TP'],
        'FP': components['FP'],
        'FN': components['FN'],
        'TN': components['TN']
    })

# Afficher les résultats
results_df = pd.DataFrame(results)
print(results_df[['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Profit (€)']].to_string(index=False))
print("")

# Identifier le meilleur selon différents critères
best_accuracy_idx = results_df['Accuracy'].idxmax()
best_f1_idx = results_df['F1-Score'].idxmax()
best_profit_idx = results_df['Profit (€)'].idxmax()

print("="*70)
print("COMPARAISON DES CRITÈRES")
print("="*70)
print("")

print(f"🎯 Meilleur Accuracy : {results_df.loc[best_accuracy_idx, 'Model']}")
print(f"   Accuracy = {results_df.loc[best_accuracy_idx, 'Accuracy']:.3f}")
print(f"   Profit = {results_df.loc[best_accuracy_idx, 'Profit (€)']:.0f}€")
print("")

print(f"🎯 Meilleur F1-Score : {results_df.loc[best_f1_idx, 'Model']}")
print(f"   F1-Score = {results_df.loc[best_f1_idx, 'F1-Score']:.3f}")
print(f"   Profit = {results_df.loc[best_f1_idx, 'Profit (€)']:.0f}€")
print("")

print(f"💰 MEILLEUR PROFIT (MÉTRIQUE MÉTIER) : {results_df.loc[best_profit_idx, 'Model']}")
print(f"   Profit = {results_df.loc[best_profit_idx, 'Profit (€)']:.0f}€")
print(f"   Accuracy = {results_df.loc[best_profit_idx, 'Accuracy']:.3f}")
print(f"   F1-Score = {results_df.loc[best_profit_idx, 'F1-Score']:.3f}")
print("")

# Analyse détaillée du meilleur modèle selon le profit
best_model = results_df.loc[best_profit_idx]
print(f"Détails du modèle optimal ({best_model['Model']}) :")
print(f"   TP (ventes réussies détectées) : {best_model['TP']:.0f} → +{best_model['TP']*10:.0f}€")
print(f"   FP (retours non anticipés) : {best_model['FP']:.0f} → -{best_model['FP']*20:.0f}€")
print(f"   FN (opportunités manquées) : {best_model['FN']:.0f} → 0€")
print(f"   TN (retours bien évités) : {best_model['TN']:.0f} → 0€")
print(f"   PROFIT TOTAL : {best_model['Profit (€)']:.0f}€")
print("")

# Visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# a) Comparaison Profit vs Métriques classiques
ax1 = axes[0, 0]
x = np.arange(len(results_df))
ax1.bar(x, results_df['Profit (€)'], alpha=0.7, edgecolor='black')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax1.set_ylabel('Profit (€)')
ax1.set_title('Profit Total par Modèle')
ax1.axhline(0, color='red', linestyle='--', linewidth=2)
ax1.grid(axis='y', alpha=0.3)

# Ajouter les valeurs
for i, v in enumerate(results_df['Profit (€)']):
    ax1.text(i, v + 50, f'{v:.0f}€', ha='center', fontweight='bold')

# b) Profit vs F1-Score (scatter)
ax2 = axes[0, 1]
scatter = ax2.scatter(results_df['F1-Score'], results_df['Profit (€)'],
                      s=100, alpha=0.7, edgecolors='black', linewidth=2)
ax2.set_xlabel('F1-Score')
ax2.set_ylabel('Profit (€)')
ax2.set_title('Profit vs F1-Score')
ax2.grid(alpha=0.3)

# Annoter les points
for idx, row in results_df.iterrows():
    ax2.annotate(row['Model'], (row['F1-Score'], row['Profit (€)']),
                fontsize=8, alpha=0.7)

# c) Comparaison multi-métriques
ax3 = axes[1, 0]
metrics_to_compare = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x_pos = np.arange(len(results_df))
width = 0.2

for i, metric in enumerate(metrics_to_compare):
    ax3.bar(x_pos + i*width, results_df[metric], width, label=metric, alpha=0.8)

ax3.set_xticks(x_pos + width * 1.5)
ax3.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax3.set_ylabel('Score')
ax3.set_title('Métriques Classiques par Modèle')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# d) Heatmap de corrélation entre métriques et profit
ax4 = axes[1, 1]
corr_data = results_df[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Profit (€)']].corr()
sns.heatmap(corr_data, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax4,
            vmin=-1, vmax=1, square=True, linewidths=1)
ax4.set_title('Corrélation entre Métriques et Profit')

plt.tight_layout()
plt.show()

# Conclusion
print("\n" + "="*70)
print("CONCLUSION")
print("="*70)
print("")
print("🔍 OBSERVATIONS IMPORTANTES :")
print("")
print("1. Le modèle avec le meilleur F1-Score N'EST PAS forcément celui avec le meilleur profit")
print("")
print("2. La métrique métier (profit) reflète mieux les objectifs business :")
print("   - Minimiser les FP (retours non détectés) est CRITIQUE (coût -20€)")
print("   - Les FN (opportunités manquées) ont moins d'impact (coût 0€)")
print("   → Un modèle plus conservateur (haute Precision) peut être préférable")
print("")
print("3. Les métriques classiques supposent des coûts égaux pour FP et FN")
print("   → Inadapté quand les erreurs ont des coûts asymétriques")
print("")
print("💡 RECOMMANDATIONS :")
print("")
print("✅ TOUJOURS définir une métrique alignée avec les objectifs métier")
print("✅ Quantifier les coûts réels de chaque type d'erreur (FP vs FN)")
print("✅ Optimiser directement cette métrique métier")
print("✅ Utiliser les métriques classiques pour le diagnostic, pas la décision finale")
print("")
print(f"🏆 DÉCISION FINALE : Choisir {results_df.loc[best_profit_idx, 'Model']}")
print(f"   → Maximise le profit : {results_df.loc[best_profit_idx, 'Profit (€)']:.0f}€")

---

## Résumé et Points Clés

À la fin de ces exercices, vous devriez être capable de :

✅ Calculer et interpréter toutes les métriques de classification

✅ Comprendre le compromis Precision vs Recall

✅ Utiliser la courbe ROC pour choisir un seuil optimal

✅ Évaluer un modèle de régression avec MSE, MAE, R²

✅ Appliquer la validation croisée pour une évaluation robuste

✅ Adapter votre choix de métriques au contexte métier

✅ Gérer les datasets déséquilibrés

---

**Prochaines étapes** :
- Chapitre 03 : Régression Linéaire et Régularisation
- Chapitre 04 : Classification Supervisée Avancée